## Contexto

El dataset Medical Cost Personal Dataset, disponible públicamente en Kaggle, recopila información de pacientes y sus costes médicos en EE.UU. A partir de variables como edad, índice de masa corporal (IMC), tabaquismo, sexo, número de hijos o región, podemos tratar de predecir diferentes objetivos relacionados con los costes sanitarios.

En esta actividad, convertiremos el problema en una tarea de clasificación binaria: predecir si una persona tiene un coste médico alto o bajo a partir de las variables disponibles.


## Descripción de las variables:


Cada fila representa una persona asegurada. Las variables disponibles son:

  * age: edad del paciente

  * sex: sexo (categoría: male, female)

  * bmi: índice de masa corporal

  * children: número de hijos a cargo

  * smoker: si es fumador o no (categoría: yes, no)

  * region: zona geográfica (categoría: northeast, northwest, southeast, southwest)

  * charges: coste total del seguro médico

# Preparación del Entorno

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier


##Carga del dataset

In [16]:
#Cargo la base de covid
df = pd.read_csv('https://raw.githubusercontent.com/gustavox0/Tecnicas_ML/refs/heads/main/insurance.csv', sep = ',')
print(df)

      age     sex     bmi  children smoker     region      charges
0      19  female  27.900         0    yes  southwest  16884.92400
1      18    male  33.770         1     no  southeast   1725.55230
2      28    male  33.000         3     no  southeast   4449.46200
3      33    male  22.705         0     no  northwest  21984.47061
4      32    male  28.880         0     no  northwest   3866.85520
...   ...     ...     ...       ...    ...        ...          ...
1333   50    male  30.970         3     no  northwest  10600.54830
1334   18  female  31.920         0     no  northeast   2205.98080
1335   18  female  36.850         0     no  southeast   1629.83350
1336   21  female  25.800         0     no  southwest   2007.94500
1337   61  female  29.070         0    yes  northwest  29141.36030

[1338 rows x 7 columns]


## Preparación del dataset

  - Crear una nueva variable high_cost que tome el valor 1 si el coste (charges) es superior a 16.000 (aproximadamente el percentil 75), y 0 en caso contrario.
  - El objetivo es aplicar el algoritmo de ÁRBOLES de DECISIÓN para predecir si una persona pertenece o no al grupo de alto coste.

In [17]:
dataset = pd.DataFrame(df)
dataset.head()


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [5]:
dataset.shape


(1338, 7)

In [19]:
#Creación de nueva variable high_cost que tome el valor 1 si el coste (charges) es superior a 16.000 y 0 en caso contrario.
dataset['high_cost'] = np.where(dataset['charges'] > 16000, 1, 0)

# Ejercicios

1. Exploración inicial del dataset

    * Análisis básico del contenido y distribución de variables.

    * Cálculo del valor de corte (percentil 75) y creación de la variable high_cost.


In [20]:
#Análisis básico del contenido y distribución de variables
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        1338 non-null   int64  
 1   sex        1338 non-null   object 
 2   bmi        1338 non-null   float64
 3   children   1338 non-null   int64  
 4   smoker     1338 non-null   object 
 5   region     1338 non-null   object 
 6   charges    1338 non-null   float64
 7   high_cost  1338 non-null   int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 83.8+ KB


In [21]:
#Revisar si hay valores NA
dataset.isnull().sum()


,0
age,0
sex,0
bmi,0
children,0
smoker,0
region,0
charges,0
high_cost,0


2. Preprocesamiento

  * Conversión de variables categóricas (sex, smoker, region) a variables numéricas (por ejemplo, con LabelEncoder o OneHotEncoder).

  * Escalado opcional de variables numéricas si se considera necesario.


In [38]:
#Conversión de variables categóricas (sex, smoker, region) a variables numéricas.
#
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
DATA=dataset.copy()
DATA['sex'] = le.fit_transform(DATA['sex']) # sex: 0=female, 1=male
DATA['smoker'] = le.fit_transform(DATA['smoker']) # smoker: 0=no, 1=si
DATA['region'] = le.fit_transform(DATA['region']) # region: 0=northeast, 1=northwest, 2=southeast, 3=southwest
DATA.head()

,age,sex,bmi,children,smoker,region,charges,high_cost
0,19,0,27.900,0,1,3,16884.92400,1
1,18,1,33.770,1,0,2,1725.55230,0
2,28,1,33.000,3,0,2,4449.46200,0
3,33,1,22.705,0,0,1,21984.47061,1
4,32,1,28.880,0,0,1,3866.85520,0


3. Entrenamiento del modelo

  * Separar datos en train/test.

  * Entrenar un modelo de árbol de decisión (DecisionTreeClassifier).

  * Probar diferentes hiperparámetros (max_depth, min_samples_split, etc.).



In [49]:
#separar datos en train/test siguiendo una proporción 70/30.
from sklearn.model_selection import train_test_split
x = DATA.iloc[:,0:-1]
y = DATA.iloc[:,-1]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
print (x_train.shape, y_train.shape)
print (x_test.shape, y_test.shape)


(936, 7) (936,)
(402, 7) (402,)


In [52]:
#Entrenar un modelo de árbol de decisión (DecisionTreeClassifier)
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, mean_absolute_error, mean_squared_error, r2_score

maxdepth=3
md_list = []
score_train_list = []
score_test_list = []
mse_train_list = []
mse_test_list = []


for maxdepth in list(range(1,7)):
  tree_model = DecisionTreeRegressor(random_state=42, max_depth=maxdepth)
  tree_model.fit(x_train, y_train)

  pred_train = tree_model.predict(x_train)
  pred_test = tree_model.predict(x_test)

  ## Generación de listas
  md_list.append(maxdepth)
  score_train_list.append(100.0*tree_model.score(x_train, y_train))
  score_test_list.append(100.0*tree_model.score(x_test, y_test))
  mse_train_list.append(mean_squared_error(y_train, pred_train))
  mse_test_list.append(mean_squared_error(y_test, pred_test))

df_aux = pd.DataFrame({"max_depth":md_list,
                   "score_train":score_train_list,
                   "score_test":score_test_list,
                   "mse_train":mse_train_list,
                   "mse_test":mse_test_list
                   })
df_aux.set_index("max_depth", inplace=True)




Evaluación en función del **MSE**

In [ ]:
#Probar diferentes hiperparámetros (max_depth, min_samples_split, etc.).


4. Visualización

  * Visualizar el árbol usando plot_tree o graphviz.

  * Representar gráficamente la importancia de las variables.



5. Evaluación

  * Evaluar con matriz de confusión, accuracy, precision, recall, f1-score.

  * Analizar posibles problemas de sobreajuste o infraajuste.




6. Explicabilidad

  * Usar SHAP (o alternativamente sklearn.inspection) para interpretar el modelo.
  * Explicar por qué el modelo toma ciertas decisiones para algunos casos concretos.


7. Predicción de nuevos casos

  * Crear dos perfiles hipotéticos de pacientes.

  * Estimar si pertenecerían o no al grupo de alto coste.
  * Justificar con el modelo entrenado qué variables han influido más.––